In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from tqdm.notebook import tqdm

# Problem: Fraud Detection

- Fraudsters create fake transactions to boost sales/shop ratings.
- Fake transactions are defined as transactions where the buyer and seller are the same individual (in reality).
- To help Shopee tackle this issue, you are expected to detect these fake transactions from normal transactions.
- Sample data for transactions and users' details will be provided.

In [2]:
devices = pd.read_csv('/kaggle/input/ptr-rd2-ahy/devices.csv', low_memory=False)
bank_acc = pd.read_csv('/kaggle/input/ptr-rd2-ahy/bank_accounts.csv', low_memory=False)
credit_cards = pd.read_csv('/kaggle/input/ptr-rd2-ahy/credit_cards.csv', low_memory=False)
orders = pd.read_csv('/kaggle/input/ptr-rd2-ahy/orders.csv', low_memory=False)

print('devices shape:', devices.shape)
print('bank accounts shape:', bank_acc.shape)
print('credit cards shape:', credit_cards.shape)
print('orders shape:', orders.shape)

devices shape: (1430311, 2)
bank accounts shape: (350841, 2)
credit cards shape: (38708, 2)
orders shape: (620947, 3)


In [3]:
devices.head()

,userid,device
0,10499978,0Zu/mWZ4cFsvobYglgZEc5VSxwwajRwrm74pBuwnrTu=
1,54526742,zBMRKyz98iy5Z7sh+JIOdF612J1CB8ggvFBI6MP361q=
2,33152428,KXC0oY3MMrTG+bnlFFvBPXZC5PW8iw6kgWztENWdIJg=
3,55020986,anRXS7+zVgCGKz5C9X3dzewIhSdojGEi5TO07pb2y+K=
4,100471502,hbIARr+USwvjjzc8QUVtD5a/apR1DJX6P3fYvgDL4gW=


In [4]:
bank_acc.head()

,userid,bank_account
0,21829134,923302000003892
1,95910542,011002023212822
2,96941876,0189303223
3,23452396,2280003199803
4,12647942,032002028484803


In [5]:
credit_cards.head()

,userid,credit_card
0,2579938,832299xxxxxx4902|02-2019
1,2154902,322429xxxxxx3848|05-2020
2,5578604,322429xxxxxx1293|11-2016
3,2051728,388329xxxxxx4303|09-2016
4,6216212,322421xxxxxx4238|05-2018


In [6]:
orders.head()

,orderid,buyer_userid,seller_userid
0,1953278092,47388162,20822974
1,1953295120,26855196,16416890
2,1953306402,121296714,28477978
3,1953314712,131221930,72837912
4,1953381964,183398314,28423332


* Each userid represents a distinct user on Shopee.
* Each orderid represents a distinct transaction on Shopee.

## Task

> ### Find fake orders where the buyer and the seller share the same details
> ### i.e. directly linked by any of the following links: Device, Credit Card, Bank Account.

In [7]:
final_df = devices.merge(bank_acc, on='userid', how='left').merge(credit_cards, on='userid', how='left')

final_df.shape

(2063983, 4)

In [9]:
final_df.head(10)

,userid,device,bank_account,credit_card
0,10499978,0Zu/mWZ4cFsvobYglgZEc5VSxwwajRwrm74pBuwnrTu=,3242920139,393922xxxxxx4431|11-2021
1,54526742,zBMRKyz98iy5Z7sh+JIOdF612J1CB8ggvFBI6MP361q=,9380814192,NaN
2,33152428,KXC0oY3MMrTG+bnlFFvBPXZC5PW8iw6kgWztENWdIJg=,0182401830,320808xxxxxx3923|07-2020
3,33152428,KXC0oY3MMrTG+bnlFFvBPXZC5PW8iw6kgWztENWdIJg=,0182401830,322421xxxxxx9202|04-2020
4,33152428,KXC0oY3MMrTG+bnlFFvBPXZC5PW8iw6kgWztENWdIJg=,9120942033,320808xxxxxx3923|07-2020
5,33152428,KXC0oY3MMrTG+bnlFFvBPXZC5PW8iw6kgWztENWdIJg=,9120942033,322421xxxxxx9202|04-2020
6,55020986,anRXS7+zVgCGKz5C9X3dzewIhSdojGEi5TO07pb2y+K=,9338029212,NaN
7,55020986,anRXS7+zVgCGKz5C9X3dzewIhSdojGEi5TO07pb2y+K=,4930029201,NaN
8,100471502,hbIARr+USwvjjzc8QUVtD5a/apR1DJX6P3fYvgDL4gW=,NaN,NaN
9,61471588,iVuRAqjJYPDrwq7W/OVP9EuiCLyPgOT9cv7ZmjDy4Gy=,391102023189822,NaN


- Nowadays, it's normal to have more than 1 smartphone right?
- We can use Shopee-Pay, virtual account even cash on the spot to pay, so not all users will enter their bank account number & use credit card

In [27]:
print('device unique values:', final_df['device'].nunique())
print('bank account unique values:', final_df['bank_account'].nunique())
print('credit card unique values:', final_df['credit_card'].nunique())

device unique values: 1363287
bank account unique values: 326606
credit card unique values: 37363


> ### That's why bank-account unique values only 24% from device unique values
> ### Credit card unique values only 2.75% from device unique values..

In [11]:
def fraud_detection(user_id_a, user_id_b):
    # buyer user
    storage_a = []

    for device in final_df[final_df['userid'] == user_id_a].device:
        if device not in storage_a:
            storage_a.append(device)
        
    for account in final_df[final_df['userid'] == user_id_a].bank_account:
        if (account not in storage_a) and (pd.notnull(account) == True):
            storage_a.append(account)
        
    for cc in final_df[final_df['userid'] == user_id_a].credit_card:
        if (cc not in storage_a) and (pd.notnull(cc) == True):
            storage_a.append(cc)
    
    # seller user
    storage_b = []

    for device in final_df[final_df['userid'] == user_id_b].device:
        if device not in storage_b:
            storage_b.append(device)
        
    for account in final_df[final_df['userid'] == user_id_b].bank_account:
        if (account not in storage_b) and (pd.notnull(account) == True):
            storage_b.append(account)
        
    for cc in final_df[final_df['userid'] == user_id_b].credit_card:
        if (cc not in storage_b) and (pd.notnull(cc) == True):
            storage_b.append(cc)
    
    # convert to pandas series
    storage_a_series = pd.Series(storage_a)
    storage_b_series = pd.Series(storage_b)
    
    result = storage_a_series.isin(storage_b_series).sum()
    
    if result > 0:
        return 1
    
    elif result == 0:
        return 0

In [12]:
orders['buyer_seller'] = list(zip(orders['buyer_userid'], orders['seller_userid']))

In [13]:
orders.head()

,orderid,buyer_userid,seller_userid,buyer_seller
0,1953278092,47388162,20822974,"(47388162, 20822974)"
1,1953295120,26855196,16416890,"(26855196, 16416890)"
2,1953306402,121296714,28477978,"(121296714, 28477978)"
3,1953314712,131221930,72837912,"(131221930, 72837912)"
4,1953381964,183398314,28423332,"(183398314, 28423332)"


In [14]:
fraud_detection(26855196, 16416890)

0

## It will take approximate 3 hours to predict all buyer-seller is fraud or not

In [16]:
is_fraud = []

for buyer, seller in tqdm(orders['buyer_seller']):
    result = fraud_detection(buyer, seller)
    is_fraud.append(result)

In [19]:
orders['is_fraud'] = is_fraud

In [22]:
orders['is_fraud'].value_counts()

0    620623
1       324
Name: is_fraud, dtype: int64

In [25]:
orders[['orderid', 'is_fraud']]

,orderid,is_fraud
0,1953278092,0
1,1953295120,0
2,1953306402,0
3,1953314712,0
4,1953381964,0
...,...,...
620942,1956636054,0
620943,1956642760,0
620944,1956651496,0
620945,1956660812,0


In [26]:
orders[['orderid', 'is_fraud']].to_csv('submission2.csv', index=False)  # remember to remove index in submission..

Name | Submitted | Wait time | Execution time | Score

submission2.csv | a minute ago | 0 seconds | 3 seconds | 1.00000